# Demonstrate the MMCController ophyd support

In [1]:
%run -i mmc_positioner.py
mmc = MMC_Positioner(name="mmc", mmc_label="is this used?")
demonstrate_mmc_positioner(mmc)

mmc = MMC_Positioner(name='mmc', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='MMC 2D XY Stage Controller')
st = MoveStatus(done=False, pos=mmc, elapsed=1.0, success=False, settle_time=0.0)
mmc = MMC_Positioner(name='mmc', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='MMC 2D XY Stage Controller')
mmc.get() = (2.1, -0.1)
exc = AttributeError("'MMC_Positioner' object has no attribute 'put'")
mmc.read() = OrderedDict([('mmc', {'value': (2.1, -0.1), 'timestamp': 1619898095.402105})])
mmc.move((.6, -.7)) = MoveStatus(done=True, pos=mmc, elapsed=1.0, success=True, settle_time=0.0)
1.015625s
mmc = MMC_Positioner(name='mmc', settle_time=0.0, timeout=None, egu='', limits=(0, 0), source='MMC 2D XY Stage Controller')
mmc.position = (0.6, -0.7)


In [2]:
from ophyd import Component
from ophyd import PseudoPositioner
from ophyd import PseudoSingle
from ophyd.pseudopos import pseudo_position_argument
from ophyd.pseudopos import real_position_argument

class TwoD_XY_StagePositioner(PseudoPositioner):

    # The pseudo positioner axes:
    x = Component(PseudoSingle, target_initial_position=True)
    y = Component(PseudoSingle, target_initial_position=True)
    
    # The real (or physical) positioners:
    pair = Component(MMC_Positioner)
    
    @pseudo_position_argument
    def forward(self, pseudo_pos):
        """Run a forward (pseudo -> real) calculation (return pair)."""
        return self.RealPosition(pseudo_pos)

    @real_position_argument
    def inverse(self, real_pos):
        """Run an inverse (real -> pseudo) calculation (return x & y)."""
        # FIXME: cannot call like this yet:  inverse([1,2])  : too many args
        pair = real_pos
        return self.PseudoPosition(x=pair[0], y=pair[-1])

In [3]:
stage = TwoD_XY_StagePositioner(name="xy_stage")
# xy_stage.inverse(xy_stage.real_position)
stage.move(8,9)

MoveStatus(done=True, pos=xy_stage, elapsed=1.0, success=True, settle_time=0.0)

In [4]:
print(f"{stage.forward((10,1))}")

TwoD_XY_StagePositionerRealPos(pair=TwoD_XY_StagePositionerPseudoPos(x=10, y=1))


In [5]:
# FIXME: x & Y are not set correctly
print(f"{stage.inverse(stage.RealPosition([1.1, 2.2]))}")

TwoD_XY_StagePositionerPseudoPos(x=[1.1, 2.2], y=[1.1, 2.2])


In [6]:
print(f"{stage.pair.move((3, 4)) = }")

stage.pair.move((3, 4)) = MoveStatus(done=True, pos=xy_stage_pair, elapsed=1.0, success=True, settle_time=0.0)


In [7]:
# FIXME: pair is OK, x & y are unset
print(f"{stage.x.get() = }")
print(f"{stage.y.get() = }")
print(f"{stage.pair.get() = }")

stage.x.get() = PseudoSingleTuple(readback=None, setpoint=8)
stage.y.get() = PseudoSingleTuple(readback=None, setpoint=9)
stage.pair.get() = (3, 4)


In [8]:
print(f"{stage.x.move(1) = }")

# FIXME: pair, x & y setpoints are OK, x & y readbacks are unset
print(f"{stage.x.get() = }")
print(f"{stage.y.get() = }")
print(f"{stage.pair.get() = }")
stage.read()

stage.x.move(1) = MoveStatus(done=True, pos=xy_stage, elapsed=1.0, success=True, settle_time=0.0)
stage.x.get() = PseudoSingleTuple(readback=None, setpoint=1)
stage.y.get() = PseudoSingleTuple(readback=None, setpoint=9)
stage.pair.get() = TwoD_XY_StagePositionerPseudoPos(x=1, y=9)


OrderedDict([('xy_stage_x', {'value': None, 'timestamp': 1619898096.480033}),
             ('xy_stage_x_setpoint',
              {'value': 1, 'timestamp': 1619898096.480033}),
             ('xy_stage_y', {'value': None, 'timestamp': 1619898096.480033}),
             ('xy_stage_y_setpoint',
              {'value': 9, 'timestamp': 1619898096.480033}),
             ('xy_stage_pair',
              {'value': TwoD_XY_StagePositionerPseudoPos(x=1, y=9),
               'timestamp': 1619898099.7308714})])

In [9]:
print(f"{stage.y.move(2.71) = }")
stage.read()

stage.y.move(2.71) = MoveStatus(done=True, pos=xy_stage, elapsed=1.0, success=True, settle_time=0.0)


OrderedDict([('xy_stage_x', {'value': None, 'timestamp': 1619898096.480033}),
             ('xy_stage_x_setpoint',
              {'value': 1, 'timestamp': 1619898096.480033}),
             ('xy_stage_y', {'value': None, 'timestamp': 1619898096.480033}),
             ('xy_stage_y_setpoint',
              {'value': 2.71, 'timestamp': 1619898096.480033}),
             ('xy_stage_pair',
              {'value': TwoD_XY_StagePositionerPseudoPos(x=1, y=2.71),
               'timestamp': 1619898100.7807584})])